In [1]:
#basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#libraries for model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score  #useful for evaluating performance of probabilistic classifiers
from imblearn.over_sampling import SMOTE 
#classification models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('modified_bail_dataset.csv')
df.head()

,Unnamed: 0,Crime_Type,Charges_Filed,Time_Served_Months,Prior_Criminal_History,Risk_of_Flight,Influence_on_Trial,Bail_Decision,Socio_Economic_Status
0,0,Cyber Crime,"['IPC 379', 'IPC 307', 'IPC 302']",33,0,0.945674,0.115675,Rejected,Low
1,1,Drug-related,"['Narcotics Act', 'Narcotics Act']",7,0,0.408927,0.946264,Granted,Low
2,2,Economic,['Cyber Laws'],41,0,0.661970,0.733362,Granted,High
3,3,Drug-related,['IPC 307'],8,0,0.464855,0.637499,Granted,High
4,4,Drug-related,"['Economic Offenses Act', 'Narcotics Act', 'IP...",37,1,0.630589,0.645278,Granted,Low


In [3]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
X = df.drop(columns=['Bail_Decision'],axis=1)

In [5]:
y = df['Bail_Decision']

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns


numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(sparse_output =False)
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((1600, 359), (400, 359))

In [9]:
if not isinstance(X_train, pd.DataFrame):
    X_train = pd.DataFrame(X_train)
print("Columns in X_train:", X_train.columns)

Columns in X_train: RangeIndex(start=0, stop=359, step=1)


Creating an Evalutation Functionn to give all metrics after model training

In [10]:
def evaluate_model(true, predicted, predicted_proba):
    accuracy = accuracy_score(true, predicted)
    precision = precision_score(true, predicted, pos_label='Granted')
    recall = recall_score(true, predicted, pos_label='Granted')
    f1 = f1_score(true, predicted, pos_label='Granted')
    roc_auc = roc_auc_score(true, predicted_proba) if predicted_proba  is not None else None
    return accuracy, precision, recall, f1, roc_auc

In [11]:
param_grids = {
    "Random Forest Classifier": {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [2, 10]
    },
    "Logistic Regression": {
        'C': [0.01, 0.1, 1.0],
        'penalty': ['l2'],
        'solver': ['lbfgs']
    },
    "Gradient Boosting Classifier": {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5]
    },
    "SVC": {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear']
    }
}

In [12]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest Classifier": RandomForestClassifier(),
    "Decision Tree Classifier": DecisionTreeClassifier(),
    "Support Vector Classifier": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Ada Boost Classifier": AdaBoostClassifier(),
    "LightGBM": LGBMClassifier(),
    "Naive bayes Classifier": GaussianNB(),
    "Extra Tree Classifier": ExtraTreesClassifier(),
    "Bagging Classifier": BaggingClassifier(),
    "Ridge Classifier": RidgeClassifier(),
    "Neural Network MLPClassifier": MLPClassifier()
}
model_list = []
f1_list = []

for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    if model_name in param_grids:
        grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='f1', n_jobs=-1)
        model = grid_search.fit(X_train, y_train).best_estimator_
    else:
        model.fit(X_train, y_train)
    
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    print(f"Cross-validated Fq Score (Training): {scores.mean():.4f}")

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_train_proba = model.predict_proba(X_train)[:,1]
        y_test_proba = model.predict_proba(X_test)[:,1]
    else:
        y_train_proba = None
        y_test_proba = None

    model_train_accuracy, model_train_precision, model_train_recall, model_train_f1, model_train_roc_auc = evaluate_model(y_train, y_train_pred, y_train_proba)
    model_test_accuracy, model_test_precision, model_test_recall, model_test_f1, model_test_roc_auc = evaluate_model(y_test, y_test_pred,y_test_proba)
    
    print(model_name)
    model_list.append(model_name)

    print("Model performance for Training set")
    print("-Accuracy: {:.4f}".format(model_train_accuracy))
    print("-Precision: {:.4f}".format(model_train_precision))
    print("-Recall: {:.4f}".format(model_train_recall))
    print("-f1 Score: {:.4f}".format(model_train_f1))
    if model_train_roc_auc is not None:
        print("-ROC AUC: {:.4f}".format(model_train_roc_auc))
    
    print('----------------------------------')

    print("Model performance for Test set")
    print("-Accuracy: {:.4f}".format(model_test_accuracy))
    print("-Precision: {:.4f}".format(model_test_precision))
    print("-Recall: {:.4f}".format(model_test_recall))
    print("-f1 Score: {:.4f}".format(model_test_f1))
    if model_test_roc_auc is not None:
        print("-ROC AUC {:.4f}".format(model_test_roc_auc))
    f1_list.append(model_test_f1)

    print('='*35)
    print('\n')




Training Logistic Regression...


c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
                ^^

Cross-validated Fq Score (Training): nan
Logistic Regression
Model performance for Training set
-Accuracy: 0.5481
-Precision: 0.5431
-Recall: 0.7857
-f1 Score: 0.6423
-ROC AUC: 0.5717
----------------------------------
Model performance for Test set
-Accuracy: 0.5375
-Precision: 0.5238
-Recall: 0.7739
-f1 Score: 0.6247
-ROC AUC 0.5252


Training Random Forest Classifier...


c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values

Cross-validated Fq Score (Training): nan
Random Forest Classifier
Model performance for Training set
-Accuracy: 0.7481
-Precision: 0.6828
-Recall: 0.9564
-f1 Score: 0.7968
-ROC AUC: 0.8792
----------------------------------
Model performance for Test set
-Accuracy: 0.4750
-Precision: 0.4820
-Recall: 0.7387
-f1 Score: 0.5833
-ROC AUC 0.4864


Training Decision Tree Classifier...


...

c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\utils\_response.py", line 204, in _get_response_values
    raise ValueError(
ValueError: pos_lab

Cross-validated Fq Score (Training): nan
Ridge Classifier
Model performance for Training set
-Accuracy: 0.6619
-Precision: 0.6753
-Recall: 0.6646
-f1 Score: 0.6699
----------------------------------
Model performance for Test set
-Accuracy: 0.4725
-Precision: 0.4681
-Recall: 0.4422
-f1 Score: 0.4548


Training Neural Network MLPClassifier...


c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_m

Cross-validated Fq Score (Training): nan
Neural Network MLPClassifier
Model performance for Training set
-Accuracy: 0.9450
-Precision: 0.9489
-Recall: 0.9443
-f1 Score: 0.9466
-ROC AUC: 0.9896
----------------------------------
Model performance for Test set
-Accuracy: 0.4475
-Precision: 0.4481
-Recall: 0.4774
-f1 Score: 0.4623
-ROC AUC 0.4577




c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\SUJAL GULIA\anaconda3\envs\bail_model\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call